<a href="https://colab.research.google.com/github/MarcelaMonteiroMontenegroGallo/Python/blob/master/Airflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from airflow.models import DAG
from datetime import datetime

Simple DAG definition

In [ ]:
etl_dag = DAG(
    dag_id = 'etl_pipeline',
    default_args={"start_date":2020-01-08"}
)

In [ ]:
# Define the default_args dictionary
default_args = {
  'owner': 'dsmith',
  'start_date': datetime(2020, 1, 14),
  'retries': 2
}


BashOperator


In [ ]:
from airflow.operators.bash_operator import BashOperator

In [ ]:
BashOperator(
    task_id='bash_exemaplo',
    bash_command ='runcleanup.sh',
    dag=ml_dag
)

In [ ]:
bash_task = BashOperator (task_id= 'clean_addresses',
                          bash_command= 'cat addresses.txt'| awk ="NF==10" > cleaned.txt, 
                          dag=dag)

In [ ]:
# Import the BashOperator
from airflow.operators.bash_operator import BashOperator

# Define the BashOperator 
cleanup = BashOperator(
    task_id='cleanup_task',
    # Define the bash_command
    bash_command='cleanup.sh',
    # Add the task to the dag
    dag=analytics_dag
)


In [ ]:
# Define a second operator to run the `consolidate_data.sh` script
consolidate = BashOperator(
    task_id='consolidate_task',
    bash_command='consolidate_data.sh',
    dag=analytics_dag)

# Define a final operator to execute the `push_data.sh` script
push_data = BashOperator(
    task_id='pushdata_task',
    bash_command='push_data.sh',
    dag=analytics_dag)


In [ ]:
from airflow.operators.python_operator import PythonOperator

In [ ]:
def printme():
  print("This goes in the logs!")

op_kwargs example


In [ ]:
def sleep(length_of_time):
  time.sleep(length_of_time)

In [ ]:
sleep_task = PythonOperator(
    task_id='sleep',
    python_callable= sleep,
    op_kwargs={'length_of_time':5}
    dag=example_dag
)

Email Operator


In [ ]:
from airflow.operators.email_operator import EmailOperator


In [ ]:
email_task =EmailOperator (
    task_id ='email_sales_report',
    to='sales_manager@example.com',
    subject ='Automated Sales Report',
    html_content ='Attached is lastest sales report',
    files ='latest_sales.xlsx',
    dag= example_dag
)

In [ ]:
def pull_file(URL, savepath):
    r = requests.get(URL)
    with open(savepath, 'wb') as f:
        f.write(r.content)   
    # Use the print method for logging
    print(f"File pulled from {URL} and saved to {savepath}")

from airflow.operators.python_operator import PythonOperator

# Create the task
pull_file_task = PythonOperator(
    task_id='pull_file',
    # Add the callable
    python_callable=pull_file,
    # Define the arguments
    op_kwargs={'URL':'http://dataserver/sales.json', 'savepath':'latestsales.json'},
    dag=process_sales_dag
)